CHUẨN BỊ DỮ LIỆU

In [2]:
import pandas as pd
import os

df = pd.read_csv("../task04_merge/stock_full_merge.csv")

# Loại bỏ các dòng không có date hợp lệ
df = df.dropna(subset=["date"])

# Chuyển date sang datetime
df["date"] = pd.to_datetime(df["date"])

# Tạo cột tháng
df["month"] = df["date"].dt.to_period("M").astype(str)

print("Dữ liệu sau khi xử lý thời gian:")
print(df.head(), "\n")


Dữ liệu sau khi xử lý thời gian:
  ticker       date  close_price exchange     sector   volume trade_type  \
0    VIC 2024-01-01      52000.0    UPCOM  Tai Chinh  10000.0      B-U-Y   
1    VIC 2024-01-01      52000.0    UPCOM  Tai Chinh  20000.0      B-U-Y   
2    VIC 2024-01-01      52000.0    UPCOM  Tai Chinh  10000.0        buy   
3    VIC 2024-01-01      52000.0    UPCOM  Tai Chinh  20000.0       SELL   
4    VIC 2024-01-01      52000.0    UPCOM  Tai Chinh  20000.0       sell   

     month  
0  2024-01  
1  2024-01  
2  2024-01  
3  2024-01  
4  2024-01   



 Tạo bảng pivot giá đóng cửa theo ngành × tháng

In [3]:
pivot_sector_month = pd.pivot_table(
    df,
    values="close_price",
    index="sector",
    columns="month",
    aggfunc="mean"
).round(2)
print("GIÁ ĐÓNG CỬA THEO NGÀNH x THÁNG:")
print(pivot_sector_month, "\n")


GIÁ ĐÓNG CỬA THEO NGÀNH x THÁNG:
month       2024-01   2024-02
sector                       
Bán Lẻ     51358.27  53155.17
Công Nghệ  54769.23  55000.00
Tai Chinh  52184.78  51873.68
Tài Chính  52571.43  52000.00 



 Pivot khối lượng giao dịch theo sàn × ngành

In [4]:
pivot_exchange_sector = pd.pivot_table(
    df,
    values="volume",
    index="exchange",
    columns="sector",
    aggfunc="sum",
    fill_value=0
)
print("KHỐI LƯỢNG GIAO DỊCH THEO SÀN x NGÀNH:")
print(pivot_exchange_sector, "\n")


KHỐI LƯỢNG GIAO DỊCH THEO SÀN x NGÀNH:
sector       Bán Lẻ  Công Nghệ   Tai Chinh  Tài Chính
exchange                                             
HNX       2625000.0        0.0         0.0        0.0
HOSE      3780000.0        0.0   4000000.0        0.0
UPCOM     5060000.0  2520000.0  13860000.0  4500000.0 



 Chuyển đổi stack/unstack để hiểu MultiIndex

Cột --> Dòng

In [5]:
print("CHUYỂN TỪ CỘT --> DÒNG: ")

stacked_sector_month = pivot_sector_month.stack()

print(stacked_sector_month.head(10), "\n")
print("Kiểu index:", stacked_sector_month.index, "\n")


CHUYỂN TỪ CỘT --> DÒNG: 
sector     month  
Bán Lẻ     2024-01    51358.27
           2024-02    53155.17
Công Nghệ  2024-01    54769.23
           2024-02    55000.00
Tai Chinh  2024-01    52184.78
           2024-02    51873.68
Tài Chính  2024-01    52571.43
           2024-02    52000.00
dtype: float64 

Kiểu index: MultiIndex([(   'Bán Lẻ', '2024-01'),
            (   'Bán Lẻ', '2024-02'),
            ('Công Nghệ', '2024-01'),
            ('Công Nghệ', '2024-02'),
            ('Tai Chinh', '2024-01'),
            ('Tai Chinh', '2024-02'),
            ('Tài Chính', '2024-01'),
            ('Tài Chính', '2024-02')],
           names=['sector', 'month']) 



Dòng --> Cột

In [6]:
print("CHUYỂN TỪ DÒNG --> CỘT:")

unstacked_sector_month = stacked_sector_month.unstack()

print(unstacked_sector_month.head(), "\n")


CHUYỂN TỪ DÒNG --> CỘT:
month       2024-01   2024-02
sector                       
Bán Lẻ     51358.27  53155.17
Công Nghệ  54769.23  55000.00
Tai Chinh  52184.78  51873.68
Tài Chính  52571.43  52000.00 



 Phân tích ngành tăng trưởng mạnh nhất

In [7]:
growth_sector = pivot_sector_month.iloc[:, -1] - pivot_sector_month.iloc[:, 0]
growth_sector = growth_sector.sort_values(ascending=False)
print("NGÀNH TĂNG TRƯỞNG MẠNH NHẤT:")
print(growth_sector, "\n")


NGÀNH TĂNG TRƯỞNG MẠNH NHẤT:
sector
Bán Lẻ       1796.90
Công Nghệ     230.77
Tai Chinh    -311.10
Tài Chính    -571.43
dtype: float64 

